In [1]:
import numpy as np
import pandas as pd
from lxml import etree

In [ ]:
# Get data file locations

CONTENT_INPUT = '../../data/raw_content.json'
CONTENT_OUTPUT = '../../data/clean_content.csv'

In [ ]:
#download the taxon data from content store for all links which are taxons
content = pd.read_json(
    CONTENT_INPUT, 
    orient='table', 
    typ='frame', 
    dtype=True, 
    convert_axes=True, 
    convert_dates=True, 
    keep_default_dates=True, 
    numpy=False, 
    precise_float=False, 
    date_unit=None
)


In [ ]:
#content.head()

In [ ]:
#content['details'][0]

In [ ]:
#content['details'][50].get('body')

In [ ]:
content = content.assign(body = [d.get('body') for d in content.details])

In [ ]:
# Clean the html

def extract_text(body):
    
    r = None
    #if lxml.html.fromstring(html).find('.//*') is not None:
    # This is horribly hacky. Previously this was failing on single new line
    # characters.
    if body and body != '\n':
        tree = etree.HTML(body)
        r = tree.xpath('//text()')
        r = ' '.join(r)
        r = r.strip().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
        r = r.replace('\n', ' ').replace(',', ' ')
        r = r.lower()
        r = ' '.join(r.split())
    if not r:
        r = ' '
    return r

In [ ]:
content = content.assign(body = content['body'].apply(extract_text))
content = content.assign(description = content['description'].apply(extract_text))
content = content.assign(title = content['title'].apply(extract_text))

In [ ]:
content['combined_text'] = content['title'] + ' ' + content['description'] + ' ' + content['body']

In [ ]:
content['taxons'] = content['taxons'].where((pd.notnull(content['taxons'])), None)
#content['taxons'] = content['taxons'].fillna(value=None)

In [ ]:
content.columns

In [ ]:
content_columns = content.drop(['taxons'], axis=1).columns.values

In [ ]:
content_wide = pd.concat([content.drop('taxons', axis=1), content['taxons'].apply(pd.Series)], axis=1)

In [ ]:
content_long = pd.melt(content_wide, id_vars=content_columns, value_name='taxon')

In [ ]:
content_long.columns

In [ ]:
content_long['taxon'][4]

In [ ]:
content_wide[0:10]

In [ ]:
mask = content_long['taxon'].isnull()
content_long = content_long[~mask]

In [ ]:
content_long = content_long.assign(taxon_id = [d['content_id'] for d in content_long['taxon']])

In [ ]:
content_long = content_long.drop(['taxon'], axis=1)

In [ ]:
content_long.shape[0]

In [ ]:
content_long.head()